intruction

- Following -->[here](https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.3/doc/doc_en/recognition_en.md)

# Env

In [ ]:
!apt install python3.8
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8
!update-alternatives --set python3 /usr/bin/python3.8
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python get-pip.py
!pip --version
!python3 -m pip install setuptools==59.5.0
# !pip install virtualenv
# !python3 -m virtualenv venv


In [ ]:
!pip  install --upgrade pip

# GPU Install
# !source /conntent/venv/bin/activate; pip install "paddlepaddle-gpu>=2.3" -i https://mirror.baidu.com/pypi/simple

# CPU Install
!python -m pip install "paddlepaddle>=2.3" -i https://mirror.baidu.com/pypi/simple

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR
%cd PaddleOCR
!pip3 install -r requirements.txt

# DAta

In [ ]:
%cd /content/

In [ ]:
!pip install gdown
!gdown -O img2latex.zip --id 1G7ALqTqyuBJMfkQJIq1CdD8faKcwxTnm
!unzip img2latex.zip

In [ ]:

# Download the pre-trained model of en_PP-OCRv3
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar
# Decompress model parameters
%cd pretrain_models
!tar -xf en_PP-OCRv3_rec_train.tar && rm -rf en_PP-OCRv3_rec_train.tar

# Train

In [ ]:
#@title config
cfg = """

Global:
  debug: false
  use_gpu: false
  epoch_num: 500
  log_smooth_window: 20
  print_batch_step: 10
  save_model_dir: ./output/v3_en_mobile
  save_epoch_step: 3
  eval_batch_step: [0, 2000]
  cal_metric_during_train: true
  pretrained_model:
  checkpoints:
  save_inference_dir:
  use_visualdl: false
  infer_img: doc/imgs_words/ch/word_1.jpg
  character_dict_path: /content/PaddleOCR/content/train_data/text_rec_data/latex_dict.txt             #changle
  max_text_length: &max_text_length 25
  infer_mode: false
  use_space_char: true
  distributed: true
  save_res_path: ./output/rec/predicts_ppocrv3_en.txt


Optimizer:
  name: Adam
  beta1: 0.9
  beta2: 0.999
  lr:
    name: Cosine
    learning_rate: 0.001
    warmup_epoch: 5
  regularizer:
    name: L2
    factor: 3.0e-05


Architecture:
  model_type: rec
  algorithm: SVTR
  Transform:
  Backbone:
    name: MobileNetV1Enhance
    scale: 0.5
    last_conv_stride: [1, 2]
    last_pool_type: avg
  Head:
    name: MultiHead
    head_list:
      - CTCHead:
          Neck:
            name: svtr
            dims: 64
            depth: 2
            hidden_dims: 120
            use_guide: True
          Head:
            fc_decay: 0.00001
      - SARHead:
          enc_dim: 512
          max_text_length: *max_text_length

Loss:
  name: MultiLoss
  loss_config_list:
    - CTCLoss:
    - SARLoss:

PostProcess:  
  name: CTCLabelDecode

Metric:
  name: RecMetric
  main_indicator: acc
  ignore_space: False

Train:
  dataset:
    name: SimpleDataSet
    data_dir: /content/PaddleOCR/content/
    ext_op_transform_idx: 1
    label_file_list:
    - /content/PaddleOCR/content/train_data/text_rec_data/rec_gt_train.txt
    transforms:
    - DecodeImage:
        img_mode: BGR
        channel_first: false
    - RecConAug:
        prob: 0.5
        ext_data_num: 2
        image_shape: [48, 320, 3]
        max_text_length: *max_text_length
    - RecAug:
    - MultiLabelEncode:
    - RecResizeImg:
        image_shape: [3, 48, 320]
    - KeepKeys:
        keep_keys:
        - image
        - label_ctc
        - label_sar
        - length
        - valid_ratio
  loader:
    shuffle: true
    batch_size_per_card: 128
    drop_last: true
    num_workers: 2
Eval:
  dataset:
    name: SimpleDataSet
    data_dir: /content/PaddleOCR/content/
    label_file_list:
    - /content/PaddleOCR/content/train_data/text_rec_data/rec_gt_test.txt
    transforms:
    - DecodeImage:
        img_mode: BGR
        channel_first: false
    - MultiLabelEncode:
    - RecResizeImg:
        image_shape: [3, 48, 320]
    - KeepKeys:
        keep_keys:
        - image
        - label_ctc
        - label_sar
        - length
        - valid_ratio
  loader:
    shuffle: false
    drop_last: false
    batch_size_per_card: 128
    num_workers: 2

"""
with open("/content/cfg.yaml", "w") as f:
    f.write(cfg)

In [ ]:
%cd /content/PaddleOCR

/content/PaddleOCR


In [ ]:
# GPU training Support single card and multi-card training
# Training icdar15 English data and The training log will be automatically saved as train.log under "{save_model_dir}"

#specify the single card training(Long training time, not recommended)#configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml \
!python3 tools/train.py -c /content/cfg.yaml -o Global.pretrained_model=/content/PaddleOCR/pretrain_models/en_PP-OCRv3_rec_train/best_accuracy

#specify the card number through --gpus
# !python3 -m paddle.distributed.launch --gpus '0,1,2,3'  tools/train.py -c configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model=en_PP-OCRv3_rec_train/best_accuracy

In [ ]:
!python3 tools/train.py -c configs/rec/rec_icdar15_train.yml \
     -o Global.pretrained_model=./pretrain_models/rec_mv3_none_bilstm_ctc_v2.0_train \
     Global.use_amp=True Global.scale_loss=1024.0 Global.use_dynamic_loss_scaling=True

# Test

In [2]:
!pip install gdown
!gdown -O cropped.zip --id 12uYi8cK4yqEfd7_VWQgjek5S30otOiek
!unzip cropped.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=12uYi8cK4yqEfd7_VWQgjek5S30otOiek
To: /content/cropped.zip
100% 1.72M/1.72M [00:00<00:00, 122MB/s]
Archive:  cropped.zip
   creating: content/cropped_img/
  inflating: content/cropped_img/30-5.jpg  
  inflating: content/cropped_img/43-4.jpg  
  inflating: content/cropped_img/43-5.jpg  
  inflating: content/cropped_img/32-1.jpg  
  inflating: content/cropped_img/52-6.jpg  
  inflating: content/cropped_img/52-1.jpg  
  inflating: content/cropped_img/50-3.jpg  
  inflating: content/cropped_img/55-4.jpg  
  inflating: content/cropped_img/39-1.jpg  
  inflating: content/cropped_img/37-0.jpg  
  inflating: content/cropped_img/45-5.

In [ ]:
!python -m pip install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
!pip install "paddleocr>=2.0.4" # Recommend to use version 2.0.1+

# MISC